In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [ ]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

# data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # note: reshape right before the first fc layer
        # size(0) is the number of elements in the tensor, example:
            # a = torch.tensor([2, 3, 4, 5])
            # print(a.size(0)) # 4
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_step = len(train_loader) # total number of `i`

for epoch in range(num_epochs): # repeat the entire training `num_epochs` times
    # for each training sample
    for i, (images, labels) in enumerate(train_loader):
        # move to gpu if available
        images = images.to(device)
        labels = labels.to(device)
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels) # still a tensor so we need to use .item() when printing
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print
        if (i+1) % 100 == 0:
            print('Epoch: [{}/{}], Step[{}/{}], Loss:{:.4f}' \
                  .format(epoch+1, num_epochs, i+1, train_step, loss.item()))

Epoch: [1/5], Step[100/600], Loss:0.0577
Epoch: [1/5], Step[200/600], Loss:0.1153
Epoch: [1/5], Step[300/600], Loss:0.0186
Epoch: [1/5], Step[400/600], Loss:0.0479
Epoch: [1/5], Step[500/600], Loss:0.0494
Epoch: [1/5], Step[600/600], Loss:0.0469
Epoch: [2/5], Step[100/600], Loss:0.1550
Epoch: [2/5], Step[200/600], Loss:0.0583
Epoch: [2/5], Step[300/600], Loss:0.0502
Epoch: [2/5], Step[400/600], Loss:0.0245
Epoch: [2/5], Step[500/600], Loss:0.0266
Epoch: [2/5], Step[600/600], Loss:0.0157
Epoch: [3/5], Step[100/600], Loss:0.1141
Epoch: [3/5], Step[200/600], Loss:0.0086
Epoch: [3/5], Step[300/600], Loss:0.0052
Epoch: [3/5], Step[400/600], Loss:0.1372
Epoch: [3/5], Step[500/600], Loss:0.0143
Epoch: [3/5], Step[600/600], Loss:0.0237
Epoch: [4/5], Step[100/600], Loss:0.0102
Epoch: [4/5], Step[200/600], Loss:0.0375
Epoch: [4/5], Step[300/600], Loss:0.0612
Epoch: [4/5], Step[400/600], Loss:0.0102
Epoch: [4/5], Step[500/600], Loss:0.2241
Epoch: [4/5], Step[600/600], Loss:0.0215
Epoch: [5/5], St

In [ ]:
# Test set
model.eval() # BatchNorm uses moving mean/variance instead of mini-batch mean/variance

with torch.no_grad():
    # initialize the stats
    correct = 0
    total = 0
    # pass through testing data once
    for images, labels in test_loader:
        # again move to device first
        images = images.to(device)
        labels = labels.to(device)
        # forward once
        outputs = model(images)
        # instead of calculating loss we will get predictions
        # it's essetially outputs just reformatting imo
        _, predicted = torch.max(outputs.data, 1)
        # accumulate stats
        total += labels.size(0) # yeah again, number of elements in the tensor
        correct += (labels == predicted).sum().item() 

    # print
    print('Test accuracy on 10000 test images: {}%' \
            .format(100 * correct / total))

# save model checkpoint
torch.save(model.state_dict(), 'cnn.ckpt')

Test accuracy on 10000 test images: 98.84%
